# imports

In [3]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import cv2
import skimage.measure
from collections import Counter
from skimage.morphology import thin
from skimage.transform import resize
import skimage
import os
import copy
from scipy import ndimage as ndi

plt.rcParams["figure.figsize"] = (10,10)

def pad_images_to_same_size(images):
    """
    :param images: sequence of images
    :return: list of images padded so that all images have same width and height (max width and height are used)
    """
    if images[0].shape == images[1].shape:
        #print("NO NEED TO RESIZE")
        return images
    width_max = 0
    height_max = 0
    for img in images:
        h, w = img.shape[:2]
        width_max = max(width_max, w)
        height_max = max(height_max, h)

    images_padded = []
    for img in images:
        h, w = img.shape[:2]
        diff_vert = height_max - h
        pad_top = diff_vert//2
        pad_bottom = diff_vert - pad_top
        diff_hori = width_max - w
        pad_left = diff_hori//2
        pad_right = diff_hori - pad_left
        img_padded = cv2.copyMakeBorder(img, pad_top, pad_bottom, pad_left, pad_right, cv2.BORDER_CONSTANT, value=0)
        assert img_padded.shape[:2] == (height_max, width_max)
        images_padded.append(img_padded)

    return images_padded


def clean_names():
    # remove spaces from img names
    img_directory = r'C:\Users\ricca\Desktop\Thesis\data\images'
    masks_directory = r'C:\Users\ricca\Desktop\Thesis\data\masks'
    img_directory_dst = r'C:\Users\ricca\Desktop\Thesis\data\imagesStdNames\\'
    masks_directory_dst = r'C:\Users\ricca\Desktop\Thesis\data\masksStdNames\\'

    count = 1
    for filename in os.listdir(img_directory):
        img = np.array(Image.open(os.path.join(img_directory, filename)))
        mask_filename = filename[:filename.rfind('.')] + ".tif"
        mask = np.array(Image.open(os.path.join(masks_directory, mask_filename)))
        mask = (mask > 50) * 255

        cv2.imwrite(img_directory_dst + str(count) + ".tif", img)
        cv2.imwrite(masks_directory_dst + str(count) + ".tif", mask)
        count += 1

# FiloDetect reimplementation

In [ ]:
def triangle_th(hist, num_bins):
    # argmax but considering the mean index of (possible) multiple max values
    xmax = int(np.round(np.mean(np.argwhere(hist == np.max(hist)))))
    h = hist[xmax]

    indi = np.where(hist > (h / 10000))[0]
    fnz = indi[0]
    lnz = indi[-1]

    lspan = xmax - fnz
    rspan = lnz - xmax
    if rspan > lspan:
        lehisto = hist.T[::-1]
        a = num_bins - lnz + 1
        b = num_bins - xmax + 1
        isflip = 1
    else:
        lehisto = hist.T
        isflip = 0
        a = fnz
        b = xmax

    # compute parameters of the straight line from first non-zero to peak
    m = h / (b - a)
    x1 = np.arange(0, (b - a))
    y1 = lehisto[x1 + a - 1]
    beta = y1 + x1 / m
    x2 = beta / (m + 1 / m)
    y2 = m * x2
    L = ((y2 - y1) ** 2 + (x2 - x1) ** 2) ** 0.5

    # obtain threshold as the location of maximum L.
    level = np.argmax(L) + 1
    level = a + np.mean(level)
    
    # Flip back if necessary
    if isflip:
        level = num_bins - level+1
    
    level = level / num_bins
    return level

def triangle_threshold(img):
    # compute histogram
    histogram, bin_edges = np.histogram(img, bins=256, range=(0, 255))
    level = triangle_th(histogram, 256)
    startp = level * 256
    localr = histogram[int(startp) : int(startp) + 10]
    x1 = np.argmin(localr) + 1
    endp = startp + x1
    levelf = endp
    img = (img > levelf).astype(int)
    return img

In [ ]:
def image_split(img, thresh):
    # OPENING morphological operation
    # approximative, matlab has different internals
    size_se = int(np.ceil(thresh))# * 2 - 1
    se = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, [size_se, size_se])
    res = cv2.erode(img, se, iterations=1)
    res = cv2.dilate(res, se, iterations=1)

    img = img - res # img with just the protrusions

    # filter by keeping only the biggest protrusions by area and the "longest" ones
    cnts, _ = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    out = np.zeros(img.shape, np.uint8)
    sum_minor_axis = 0
    count_minor_axis = 0

    for c in cnts:
        contour_area = cv2.contourArea(c)
        if contour_area < thresh:
            continue

        ellipse = cv2.fitEllipse(c)
        major_axis = max(ellipse[1])
        minor_axis = min(ellipse[1])
        
        if major_axis >= minor_axis * 1.5: # KEY PARAMETER
            cv2.drawContours(out, [c], -1, 255, cv2.FILLED)
            sum_minor_axis += minor_axis
            count_minor_axis += 1
    
    # TODO handle this error, no cnts on problematic images: they get binarized in a huge white spot
    avg_minor_axis = sum_minor_axis / count_minor_axis if count_minor_axis != 0 else 0
            
    return out, avg_minor_axis

In [ ]:
def find_branchpoints(skeleton):
    #skeleton = skeleton.astype(int)
    return find_endpoints(skeleton) - 2

def find_endpoints(img):
    # Find row and column locations that are non-zero
    (rows,cols) = np.nonzero(img)

    # Initialize empty list of co-ordinates
    skel_coords = []

    # For each non-zero pixel...
    for (r,c) in zip(rows,cols):

        # Extract an 8-connected neighbourhood
        (col_neigh,row_neigh) = np.meshgrid(np.array([c-1,c,c+1]), np.array([r-1,r,r+1]))

        # Cast to int to index into image
        col_neigh = col_neigh.astype('int')
        row_neigh = row_neigh.astype('int')

        # Convert into a single 1D array and check for non-zero locations
        pix_neighbourhood = img[row_neigh,col_neigh].ravel() != 0

        # If the number of non-zero locations equals 2, add this to 
        # our list of co-ordinates
        if np.sum(pix_neighbourhood) == 2:
            skel_coords.append((r,c))

    return len(skel_coords)

def detect_fused(img):

    img_thinned = thin(img) # or skeletonize, small difference
    img_thinned[0,:] = 0
    img_thin_labeled = skimage.measure.label(img_thinned.astype(np.uint8), connectivity=2)
    img_labeled = skimage.measure.label(img.astype(np.uint8), connectivity=2)
    stats_bbox = skimage.measure.regionprops(img_thin_labeled.astype(np.uint8))
    # results to fill
    fused_image = np.zeros_like(img)
    singles_image = np.zeros_like(img)
    finish = np.zeros_like(img)

    for i in range(0, len(stats_bbox)):

        bbox = stats_bbox[i].bbox
        # take thinned branch region
        bbox_region = img_thin_labeled[bbox[0]:bbox[2], bbox[1]:bbox[3]]

        # take its largest connected component in case multiple accidentally are in that bounding box
        value_counts = Counter(bbox_region.flatten()).most_common()
        most_frequent_value = value_counts[1][0] if len(value_counts) > 1 else value_counts[0][0]
        bbox_region = (bbox_region == most_frequent_value) * 1

        # if into that bounding box #branchpoints > 1 AND #endpoints >= 4, it is a FUSED filopodia
        bbox_region_padded = np.pad(bbox_region, pad_width=4, mode='constant', constant_values=0)
        n_endpoints = find_endpoints(bbox_region_padded)
        n_branchpoints = find_branchpoints(bbox_region_padded)
        is_fused = n_branchpoints > 1 and n_endpoints >= 4

        # mark FUSED and SINGLE regions with 2 different values
        if is_fused:
            fused_image += (img_labeled == (i + 1))
        else:
            singles_image += (img_labeled == (i + 1))
        finish = singles_image + fused_image * 2

    return finish

In [ ]:
def FiloDetect(img):
    # convert to grayscale and slight blur

        if len(img.shape) > 2:
            img = np.mean(img, axis=2)[1:,:]
        
        # slight blur
        #img = cv2.filter2D(img, -1, np.ones((2, 2), np.float32) / 4)

        # threshold the image
        img = triangle_threshold(img)

        # fill the inside of the cell
        h, w = img.shape[:2]
        mask = np.zeros((h+2, w+2), np.uint8)
        img =  255 - cv2.floodFill(img, mask, (0,0), 255)[1] > 50
        img = (img * 255).astype(np.uint8)
        img_t = img.copy()

        # keep only biggest blob
        cnts, _ = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        cnt = max(cnts, key=cv2.contourArea)
        img = np.zeros(img.shape, np.uint8)
        cv2.drawContours(img, [cnt], -1, 255, cv2.FILLED)

        # extract filopodia
        img, avg_minor_axis = image_split(img, 9 * 0.6)

        # classify fused/unfused filopodia
        img_fused = detect_fused(img / 255)
        return img, img_fused

# FiloAnalyzer

In [7]:
def adjust_gamma(image, gamma=1.0):
        # build a lookup table mapping the pixel values [0, 255] to
        # their adjusted gamma values
        invGamma = 1.0 / gamma
        table = np.array([((l / 255.0) ** invGamma) * 255 for l in np.arange(0, 256)]).astype("uint8")
        # apply gamma correction using the lookup table
        #print(type(image[0,0]))
        return cv2.LUT(image.astype(np.uint8), table)

In [8]:
def FiloAnalyzer_full(img, threshold_method, erosion, dilation, filopodia_area_threshold, gamma, sigma, structuring_element, aspect_ratio):
        threshold_image = 150

        # automatic selection of cell and nuclues channel
        if len(img.shape) > 2:
            body = img[:,:,0]
        else:
            body = img
        body[body < 0.3 * np.mean((body))] = 0
        body = adjust_gamma(body, gamma=gamma)

        if threshold_method == "automatic":
            test = copy.copy(body)
            test2 = copy.copy(body)
            body = cv2.adaptiveThreshold(test, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 655, 5)
            body = cv2.morphologyEx(body, cv2.MORPH_ERODE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5)))
            body = ndi.binary_fill_holes(body).astype(np.uint8)
            x, _ = cv2.findContours(body, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
            body = np.zeros_like(body)
            for i in x:
                mom = cv2.moments(i)
                area = mom['m00']
                if area > 10000 and area < 2000000:
                    cv2.drawContours(body, [i], -1, (255, 144, 255), -1)

        elif threshold_method == "triangle":
            test2 = copy.copy(body)
            hist = np.histogram(body.ravel(), bins=256, range=(0.0, 255))
            ret, body = cv2.threshold(body, threshold_image, 255, cv2.THRESH_BINARY + cv2.THRESH_TRIANGLE)
            s = hist[0][int(ret):int(ret) + 12]
            s[np.where(s == 0)] = np.max(s)
            t = np.argmin(s) + int(ret)
            ret, body = cv2.threshold(test2, t, 255, cv2.THRESH_BINARY)
            body = ndi.binary_fill_holes(body).astype(np.uint8)

        else:
            ret, body = cv2.threshold(body, threshold_image, 255, cv2.THRESH_BINARY)
        
        thresholded_image = body.copy()
        
        
        # EROSION AND DILATION TODO provare anche morph_rect?
        nucleus = cv2.morphologyEx(body, cv2.MORPH_ERODE, cv2.getStructuringElement(structuring_element, (erosion, erosion)))
        nucleus = cv2.morphologyEx(nucleus, cv2.MORPH_DILATE, cv2.getStructuringElement(structuring_element, (dilation, dilation)))
        body[nucleus != 0] = 0

        opened_image = body.copy()

        # EXTRACT ONLY BIGGEST AND ELONGATED CONTOURS
        result = np.zeros_like(body)
        x, _ = cv2.findContours(body.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        for i in x:
            moments = cv2.moments(i)
            area = moments['m00']
            if area > filopodia_area_threshold and area < 200000:
                if moments['m00'] != 0.0:
                    ellip = cv2.fitEllipse(i)
                    (_, axes, _) = ellip
                    major_axis = max(axes)
                    minor_axis = min(axes)
                    if np.sqrt(1 - (minor_axis * minor_axis) / (major_axis * major_axis)) > aspect_ratio: # 0.7
                        cv2.drawContours(result, [i], -1, 1, -1)

        return thresholded_image, opened_image, result

In [9]:
IOUs, DICEs, PRECISIONs, RECALLs, F1SCOREs, MSEs = [],[],[],[],[],[]
filo_N_diffs, filo_N_abs_diffs, filo_len_diffs, filo_len_abs_diffs = [],[],[],[]
single_filo_N_diff, single_filo_N_abs_diff, merged_filo_N_diff, merged_filo_N_abs_diff = [],[],[],[]
img_directory = r'C:\Users\ricca\Desktop\Thesis\data\imagesStdNames'
mask_directory = r'C:\Users\ricca\Desktop\Thesis\data\masksStdNames'
#os.makedirs(r"C:\Users\ricca\Desktop\Thesis\data\FiloAnalyzerTUNED320")
plt.rcParams["figure.figsize"] = (15,15)

IMG_HEIGHT, IMG_WIDTH = 320, 320

for i in range(1, 99+1):
    filename = f"{i}.tif"
    img = cv2.imread(os.path.join(img_directory, filename))
    mask = cv2.imread(os.path.join(mask_directory, filename))

    # preprocess the image
    if len(img.shape) > 2:
        img = img.mean(axis=-1)
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    img = cv2.GaussianBlur(img,(3,3),0)
    #img = img.reshape((1, IMG_HEIGHT, IMG_WIDTH, 1))
    #img = img / 255.0

    # preprocess the mask
    if len(mask.shape) > 2:
        mask = mask.mean(axis=-1)
    mask = resize(mask, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    mask = (mask > 0.80) * 255

    thresholded_image, opened_image, pred = FiloAnalyzer_full(img, 'triangle', 16, 16, 30, 1, 1, cv2.MORPH_ELLIPSE, 0.7)
    pred = pred * 255

    #assert mask.dtype == pred.dtype

    cv2.imwrite(os.path.join(r"C:\Users\ricca\Desktop\Thesis\data\FiloAnalyzerTUNED320", f"{i}.tif"), pred)


# define metrics

In [ ]:
def iou(prediction, true_mask):
    intersection = np.logical_and(prediction, true_mask).sum()
    union = np.logical_or(prediction, true_mask).sum()
    iou_score = intersection / union
    return iou_score

def dice(prediction, true_mask):
    intersection = np.logical_and(prediction, true_mask).sum()
    dice_score = (2. * intersection) / (prediction.sum() + true_mask.sum())
    return dice_score

def precision(prediction, true_mask):
    true_positives = np.logical_and(prediction, true_mask).sum()
    false_positives = np.logical_and(prediction, np.logical_not(true_mask)).sum()
    precision_score = true_positives / (true_positives + false_positives)
    return precision_score

def recall(prediction, true_mask):
    true_positives = np.logical_and(prediction, true_mask).sum()
    false_negatives = np.logical_and(np.logical_not(prediction), true_mask).sum()
    recall_score = true_positives / (true_positives + false_negatives)
    return recall_score

def f1_score(prediction, true_mask):
    p = precision(prediction, true_mask)
    r = recall(prediction, true_mask)
    f1 = 2 * (p * r) / (p + r)
    return f1

def mse(prediction, true_mask):
    mse_score = np.mean((prediction - true_mask) ** 2)
    return mse_score

def num_filopodia_blobs(mask):
    return skimage.measure.label(mask)

def num_filopodia_demerged(mask):
    thinned = thin(mask)
    img_thin_labeled = skimage.measure.label(thinned.astype(np.uint8), connectivity=2)
    stats_bbox = skimage.measure.regionprops(img_thin_labeled.astype(np.uint8))
    filopodia_count = 0
    for i in range(0, len(stats_bbox)):
        bbox = stats_bbox[i].bbox
        bbox_region = img_thin_labeled[bbox[0]:bbox[2], bbox[1]:bbox[3]]

        value_counts = Counter(bbox_region.flatten()).most_common()
        most_frequent_value = value_counts[1][0] if len(value_counts) > 1 else value_counts[0][0]
        bbox_region = (bbox_region == most_frequent_value) * 1

        # if into that bounding box #branchpoints > 1 AND #endpoints >= 4, it is a FUSED filopodia
        bbox_region_padded = np.pad(bbox_region, pad_width=4, mode='constant', constant_values=0)
        n_endpoints = find_endpoints(bbox_region_padded)
        
        filopodia_count += n_endpoints - 1
    return filopodia_count

def filopodia_length_sum(mask):
    return np.count_nonzero(thin(mask))


# FILOANALYZER CROSSVALIDATION

In [ ]:
img_directory = r'C:\Users\ricca\Desktop\Thesis\data\imagesStdNames'
masks_directory = r'C:\Users\ricca\Desktop\Thesis\data\masksStdNames'
X, Y = [], []
for i in range(1, 90 + 1):
        img = np.array(Image.open(os.path.join(img_directory, f"{i}.tif")))
        mask = np.array(Image.open(os.path.join(masks_directory, f"{i}.tif")))
        mask = (mask > 50) * 255
        X.append(img)
        Y.append(mask)

def evaluate_params(threshold_method, erosion, dilation, filopodia_area_threshold, gamma, sigma, structuring_element, aspect_ratio):

    IOUs = []
    for img, mask in zip(X, Y):

        thresholded_image, opened_image, label = FiloAnalyzer_full(img, threshold_method, erosion, dilation, filopodia_area_threshold, gamma, sigma, structuring_element, aspect_ratio)
        # resize for occasional missing row/col
        imgs = pad_images_to_same_size([label, mask])
        label, mask = imgs[0], imgs[1]
        IOUs.append(iou(mask, label))
    return np.mean(IOUs)

In [ ]:
import cv2

In [ ]:
import itertools
import multiprocessing

param_grid = {
    'threshold_method': ["triangle"],
    'erosion': [18, 19, 20, 21, 22],
    'dilation': [18, 19, 20, 21, 22],
    'filopodia_area_threshold': [20, 30, 40],
    'gamma': [1],
    'sigma': [1],
    'structuring_element': [cv2.MORPH_ELLIPSE],
    'aspect_ratio': [0.6, 0.7, 0.8]
}
param_combinations = list(itertools.product(*param_grid.values()))
print("EXAMINING", len(param_combinations), "CONFIGURATIONS")
IOUs = np.array([0] * len(param_combinations), dtype=np.float32)

for i, params in enumerate(param_combinations):
    IOUs[i] = evaluate_params(*params)
    print(f"{i}/{len(param_combinations)}", params, IOUs[i], "BEST", np.max(IOUs), param_combinations[np.argmax(IOUs)])
print("BEST IS", np.max(IOUs), param_combinations[np.argmax(IOUs)])
# 15360 CONFIGURATIONS
# BEST FULL     0.15683042 ('triangle', 16, 16, 30, 1, 1, ellipse, 0.7)
# BEST TRAINING 0.15395133 ('triangle', 20, 20, 20, 1, 1, ellipse, 0.8)

EXAMINING 225 CONFIGURATIONS
0/225 ('triangle', 18, 18, 20, 1, 1, 2, 0.6) 0.1520575 BEST 0.1520575 ('triangle', 18, 18, 20, 1, 1, 2, 0.6)
1/225 ('triangle', 18, 18, 20, 1, 1, 2, 0.7) 0.15294771 BEST 0.15294771 ('triangle', 18, 18, 20, 1, 1, 2, 0.7)
2/225 ('triangle', 18, 18, 20, 1, 1, 2, 0.8) 0.1525439 BEST 0.15294771 ('triangle', 18, 18, 20, 1, 1, 2, 0.7)
3/225 ('triangle', 18, 18, 30, 1, 1, 2, 0.6) 0.1522353 BEST 0.15294771 ('triangle', 18, 18, 20, 1, 1, 2, 0.7)
4/225 ('triangle', 18, 18, 30, 1, 1, 2, 0.7) 0.15321764 BEST 0.15321764 ('triangle', 18, 18, 30, 1, 1, 2, 0.7)
5/225 ('triangle', 18, 18, 30, 1, 1, 2, 0.8) 0.15243663 BEST 0.15321764 ('triangle', 18, 18, 30, 1, 1, 2, 0.7)
6/225 ('triangle', 18, 18, 40, 1, 1, 2, 0.6) 0.15178667 BEST 0.15321764 ('triangle', 18, 18, 30, 1, 1, 2, 0.7)
7/225 ('triangle', 18, 18, 40, 1, 1, 2, 0.7) 0.15260369 BEST 0.15321764 ('triangle', 18, 18, 30, 1, 1, 2, 0.7)
8/225 ('triangle', 18, 18, 40, 1, 1, 2, 0.8) 0.15172608 BEST 0.15321764 ('triangle', 18

In [ ]:
import os
import shutil
import time

input_folder = r"C:\Users\ricca\Downloads\newData\2023 Lisa Data new Mask and Bio Data\Cell Masks\IMAGESTEST"
output_folder = r"C:\Users\ricca\Downloads\newData\2023 Lisa Data new Mask and Bio Data\Cell Masks\IMAGESTESTStdNames"
#input_folder = r"C:\Users\ricca\Desktop\ImgsFinal"
#output_folder = r"C:\Users\ricca\Desktop\ImgsFinalOrdered"
extension = ".tif"

# Get a list of image files in the input folder
image_files = [f for f in os.listdir(input_folder) if f.endswith(extension)]
image_files.sort(key=lambda x: os.path.getmtime(os.path.join(input_folder, x)))

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Rename and copy images to the output folder
for i, image_file in enumerate(image_files, start=1):
    old_path = os.path.join(input_folder, image_file)
    new_filename = f"{i}{extension}"
    new_path = os.path.join(output_folder, new_filename)
    shutil.copy(old_path, new_path)
    #time.sleep(1)
    print(f"Renamed {image_file} to {new_filename}")

print("Renaming and copying complete.")


Renamed 1.tif to 1.tif
Renamed 10.tif to 2.tif
Renamed 11.tif to 3.tif
Renamed 12.tif to 4.tif
Renamed 13.tif to 5.tif
Renamed 14.tif to 6.tif
Renamed 15.tif to 7.tif
Renamed 16.tif to 8.tif
Renamed 17.tif to 9.tif
Renamed 18.tif to 10.tif
Renamed 19.tif to 11.tif
Renamed 2.tif to 12.tif
Renamed 20.tif to 13.tif
Renamed 21.tif to 14.tif
Renamed 22.tif to 15.tif
Renamed 23.tif to 16.tif
Renamed 24.tif to 17.tif
Renamed 25.tif to 18.tif
Renamed 26.tif to 19.tif
Renamed 27.tif to 20.tif
Renamed 28.tif to 21.tif
Renamed 29.tif to 22.tif
Renamed 3.tif to 23.tif
Renamed 30.tif to 24.tif
Renamed 31.tif to 25.tif
Renamed 32.tif to 26.tif
Renamed 33.tif to 27.tif
Renamed 34.tif to 28.tif
Renamed 35.tif to 29.tif
Renamed 36.tif to 30.tif
Renamed 37.tif to 31.tif
Renamed 38.tif to 32.tif
Renamed 39.tif to 33.tif
Renamed 4.tif to 34.tif
Renamed 40.tif to 35.tif
Renamed 41.tif to 36.tif
Renamed 42.tif to 37.tif
Renamed 43.tif to 38.tif
Renamed 44.tif to 39.tif
Renamed 45.tif to 40.tif
Renamed 46.ti